# Short summary
### Dataset
Resized Cityscapes 1024x2048 -> 256x256. Totally 2975 training images, 500 validation images.

### Three configurations
1. *city_gen_ce_b1*: netG (J.Johnson's ResNet9blocks), netD (3-layer ConvNet), CrossEnt + GAN loss, optim (ADAM + lr =0.0002 + linear decay (i.e., lr -= const after 100 epochs)), batchSize=1, 260 epochs.
2. *city_gen_ce_lsgan_nonoise_b1*: the same as *city_gen_ce_b1*, except using LSGAN.
3. *city_gen_ce_lsgan_b1*: the same as *city_gen_ce_lsgan_nonoise_b1*, except adding noise to ground truth when computing the LSGAN loss.

### Loss
* G_Total: The total loss for the generator (i.e., the segmentation network).
* G_CE: CrossEnt loss for the generator.
* G_GAN: GAN loss for the generator, which is the terms involving the generator.
* D_GAN: GAN loss for the discriminator.


In [18]:
import parse
import argparse
import pandas as pd
import os
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
import numpy as np
from bokeh.charts.utils import cycle_colors
from bokeh.models import HoverTool, WheelPanTool, SaveTool, BoxZoomTool

output_notebook()

#parser = argparse.ArgumentParser()
#parser.add_argument('--f', required=True, type=str, help='loss log file')
#opt = parser.parse_args()
#fname = opt.f
#df = log2df(fname)

## data frame
def log2df(fname, format_strs=None):
    if format_strs is None:
        format_strs = ['epoch: {epoch:d}', 'iters: {iters:d}', 'time: {time:f}', 'G_Total: {G_Total:f}', 'G_CE: {G_CE:f}', 'G_GAN: {G_GAN:f}', 'D_GAN: {D_GAN:g}']
    columns = [r.fixed[0] for r in parse.findall('{{{}:', ''.join(format_strs))]
    df = pd.DataFrame(columns=columns)
    with open(fname, 'r') as f:
        for line in f:
            #parsed = parse.parse(format_str, line)
            #for k,v in parsed.named.items(): # only one key
            #    data.setdefault(k, []).append(v)
            if not line.startswith('(epoch'):
                continue
            data = dict()
            for fmat in format_strs:
                parsed = parse.search(fmat, line)
                if parsed is None:
                    continue
                data.update(parsed.named)
            df = df.append(data, ignore_index=True)
    return df

ckpt_dir = './checkpoints/'
legends = [d for d in os.listdir(ckpt_dir) if os.path.isdir(ckpt_dir + '/' +d)]
frames = [log2df(os.path.join(ckpt_dir,d,'loss_log.txt')) for d in legends]

Loading BokehJS ...

In [20]:
## plot
colors = ['red','blue','green','black','purple','orange','yellow','cyan']
fields_hover = frames[0].columns.tolist()
hover = HoverTool(tooltips=[(field, "@"+field) for field in fields_hover])
p = figure(title='train_loss',
           x_axis_label='iteration',
           width=1000,
           tools = [hover, WheelPanTool(), SaveTool(), BoxZoomTool()])
for i,frame in enumerate(frames):
    ds = ColumnDataSource(frame)
    p.line('index', 'G_Total', source = ds,
           color=colors[i], legend=legends[i], line_dash=[4,4])
show(p)